In [242]:
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

train = pd.read_csv('~/Desktop/kaggle/titanic/train.csv')
test = pd.read_csv('~/Desktop/kaggle/titanic/test.csv')

In [243]:
train = train.append(test)

In [244]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [245]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  1309 non-null   int64  
 1   Survived     891 non-null    float64
 2   Pclass       1309 non-null   int64  
 3   Name         1309 non-null   object 
 4   Sex          1309 non-null   object 
 5   Age          1046 non-null   float64
 6   SibSp        1309 non-null   int64  
 7   Parch        1309 non-null   int64  
 8   Ticket       1309 non-null   object 
 9   Fare         1308 non-null   float64
 10  Cabin        295 non-null    object 
 11  Embarked     1307 non-null   object 
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


In [246]:
train.isnull().any()

PassengerId    False
Survived        True
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare            True
Cabin           True
Embarked        True
dtype: bool

In [247]:
train['Age'] = train['Age'].fillna(train['Age'].mean())
train['Fare'] = train['Fare'].fillna(train['Fare'].mean())

In [248]:
train['Cabin'].value_counts()

C23 C25 C27        6
B57 B59 B63 B66    5
G6                 5
C78                4
B96 B98            4
                  ..
C50                1
C70                1
F                  1
C49                1
B102               1
Name: Cabin, Length: 186, dtype: int64

In [249]:
train['Embarked'].value_counts()

S    914
C    270
Q    123
Name: Embarked, dtype: int64

In [250]:
train['Cabin'] = train['Cabin'].fillna('U')
train['Embarked'] = train['Embarked'].fillna('S')

In [251]:
sex_map = {'male':1,'female':0}
train['Sex'] = train['Sex'].map(sex_map)

In [252]:
Embarked_df = pd.get_dummies(train['Embarked'],prefix='Embarked')

In [253]:
train = pd.concat([train,Embarked_df],axis=1)

In [254]:
Pclass_df = pd.get_dummies(train['Pclass'],prefix='Pclass')
train = pd.concat([train,Pclass_df],axis=1)

In [255]:
train['Cabin'] = train['Cabin'].map(lambda x:x[0])
Cabin_df = pd.get_dummies(train['Cabin'],prefix='Cabin')
train = pd.concat([train,Cabin_df],axis=1)

In [256]:
train.corr()['Survived'].sort_values()

Sex           -0.543351
Pclass        -0.338481
Pclass_3      -0.322308
Cabin_U       -0.316912
Embarked_S    -0.149683
Age           -0.070323
SibSp         -0.035322
Cabin_T       -0.026456
PassengerId   -0.005007
Embarked_Q     0.003650
Cabin_G        0.016040
Cabin_A        0.022287
Cabin_F        0.057935
Parch          0.081629
Pclass_2       0.093349
Cabin_C        0.114652
Cabin_E        0.145321
Cabin_D        0.150716
Embarked_C     0.168240
Cabin_B        0.175095
Fare           0.257307
Pclass_1       0.285904
Survived       1.000000
Name: Survived, dtype: float64

In [257]:
new_train = pd.concat([Embarked_df,Pclass_df,Cabin_df,train['Age'],train['Fare']],axis=1)

In [258]:
# predict_x = [[x] for x in test_X]
# test_y = [[y] for y in test_Y]

model = GaussianNB()

train_x = new_train.iloc[0:891]
train_y = train['Survived'][0:891]

test_x = new_train.iloc[891:len(new_train)]

model.fit(train_x, train_y)


prediction = model.predict(test_x)

test = pd.read_csv('~/Desktop/kaggle/titanic/test.csv')

result = {'PassengerId':list(test['PassengerId']),'Survived':list(prediction)}

result = pd.DataFrame(result)

In [266]:
result['Survived'] = result['Survived'].map(lambda x: int(x))

In [267]:
result.to_csv('result.csv',index=None)